Make sure to pip install pickle, praw, and imdbpy

In [1]:
import json
import requests
import praw
import pprint
from collections import Counter
import pickle
import os
import shutil


Pickle all data to be more easily reusable

In [2]:
r = praw.Reddit(user_agent='reddit_hype')
subreddits = r.search_reddit_names('deadpool')


with open( "pickled/saved_queries.p", "wb" ) as f:
	pickle.dump(subreddits, f)



print('Pickled')

Pickled


Print list of top 100 imdb movies

In [3]:
import tmdbsimple as tmdb
with open( "API_keys/tmdb_api_key.txt", "rb" ) as f:
     tmdb_api_key = f.read()

tmdb.API_KEY = tmdb_api_key.strip()
discover = tmdb.Discover()

all_responses = {}
for n in range(2010,2016):
    responses= []
    for x in range(1,6):
        response = discover.movie(page=x, year=n)
        responses.append(response['results'])    

    # flatten list of lists
    responses = [val for sublist in responses for val in sublist]
    all_responses[n] = responses

In [4]:
# get all dicts together
aggregate_responses = [all_responses[year_of_movies] for year_of_movies in all_responses]

# flatten
aggregate_responses = [ movie for yearlist in aggregate_responses for movie in yearlist]
all_title_dates = [(movie['title'], movie['release_date']) for movie in aggregate_responses]
all_title_dates


/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:90: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:96: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:675: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:667: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IP

[('Inception', '2010-07-14'),
 ('Avatar', '2009-12-10'),
 ('Terminator Salvation', '2009-05-20'),
 ('Shutter Island', '2010-02-18'),
 ('The Hangover', '2009-06-05'),
 ('Iron Man 2', '2010-04-28'),
 ('Harry Potter and the Deathly Hallows: Part 1', '2010-10-17'),
 ('Alice in Wonderland', '2010-03-03'),
 ('Despicable Me', '2010-07-08'),
 ('Saving Private Ryan', '1998-07-24'),
 ('The Book of Eli', '2010-01-14'),
 ('Sherlock Holmes', '2009-01-07'),
 ('Zombieland', '2009-10-07'),
 ('The Godfather: Part II', '1974-12-20'),
 ('The Godfather', '1972-03-15'),
 ('The Karate Kid', '2010-06-10'),
 ('Toy Story 3', '2010-06-16'),
 ('The Shining', '1980-05-22'),
 ('The Expendables', '2010-08-03'),
 ('How to Train Your Dragon', '2010-03-05'),
 ('Black Swan', '2010-12-02'),
 ('RED', '2010-10-13'),
 ('Salt', '2010-07-21'),
 ('Beauty and the Beast', '1991-11-12'),
 ('Tangled', '2010-11-24'),
 ("The Sorcerer's Apprentice", '2010-07-13'),
 ('Percy Jackson & the Olympians: The Lightning Thief', '2010-02-01')

Get top subreddits for all of the corresponding movies

In [5]:
r = praw.Reddit(user_agent='reddit_hype')

with open( "pickled/saved_queries.p", "rb" ) as f:
	subreddits = pickle.load(f)

print(subreddits)
print(type(subreddits[0]))



[Subreddit(subreddit_name='deadpool'), Subreddit(subreddit_name='DeadpoolENGDVDScr'), Subreddit(subreddit_name='deadpoolmodz'), Subreddit(subreddit_name='DEADPOOLforPRESIDENT'), Subreddit(subreddit_name='DeadpoolGamingForum'), Subreddit(subreddit_name='DeadpoolMovie4'), Subreddit(subreddit_name='deadpoolhate'), Subreddit(subreddit_name='deadpoolmasterrace'), Subreddit(subreddit_name='deadpoole')]
<class 'praw.objects.Subreddit'>


In [97]:
#return customized name for pickle file
def getPickleFilename(subreddit, counter):
    return "pickled/" + str(subreddit) + "/" + str(counter) + ".p"

#The input must be a praw.objects.Subreddit instance.
#This function takes all of the posts (in the form of Submission objects) of a subreddit 
#and places them in lists of lengths at most 1000. Then, it pickles those lists. This allows
#for easy access to the data from Reddit so that you never have to have the entire dataset
#in memory all at once (which may cause overflow errors).
def pickleRedditData (subreddit):
    #create the generator for the Submission objects. The empty string is for an empty query to
    #include all posts in the results
    results = subreddit.search("", limit = None)
    
    #counts how many pickle files we have written so far
    counter = 0
    
    submissions = []
    dir_name = "pickled/" + str(subreddit)
    if(os.path.isdir(dir_name)):
        shutil.rmtree(dir_name)
    os.mkdir(dir_name)
    try:
        while(True):
            for i in range(1000):
                submission = next(results)
                submissions.append(submission)
            filename = getPickleFilename(subreddit, counter)
            with open( filename, "wb" ) as f:
                pickle.dump(submissions, f)
                counter = counter + 1
    except StopIteration:
        filename = getPickleFilename(subreddit, counter)
        with open( filename, "wb" ) as f:
            pickle.dump(submissions, f)
            counter = counter + 1
        
            
r = praw.Reddit(user_agent='reddit_hype')
subreddits = r.search_reddit_names('deadpool')
subreddit = subreddits[0]
pickleRedditData(subreddit)
filename = getPickleFilename(subreddit, 0)
with open (filename, "rb") as f:
    #remember these results are just the submissions from one pickle file
    #There may be more pickle files.
    results = pickle.load(f)
    for i in range(10):
        print(results[i])
    

3473 :: Deadpool's Epic Opening
3889 :: No title needed
373 :: Sadly missed Canada day, But not today!
321 :: Weapon X
333 :: When ever I see a deadpool post on a different sub-reddit.
344 :: "Let's find out." (Spoilers) Deadpool vs Carnage #1.
322 :: Two of my favorite characters.
359 :: All Deadpool cares about is chimichangas
546 :: Ryan Reynolds stole a Deadpool suit from the set. That man was born to...
454 :: Bat-pool.


In [94]:
class Movie_Data:
    def __init__(self, release_date, rating, budget, box_office):
        self.release_date = release_date
        self.rating = rating
        if(budget == 0):
            box_office
        else:
            self.budget_ratio = box_office/budget
        self.subreddits = []
        
    def add_subreddit(self, subreddit):
        self.subreddits = self.subreddits + [subreddit]
    
    def add_subreddits(self, subreddits):
        self.subreddits = self.subreddits + subreddits

['hi']

In [87]:
results[0].id

sub = r.get_submission(submission_id = "468qqi")
#Here are the attributes of the submission objects
dir(sub._comments[0]._replies[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_fast_permalink',
 '_get_json_dict',
 '_has_fetched',
 '_has_fetched_replies',
 '_info_url',
 '_populate',
 '_post_populate',
 '_replies',
 '_submission',
 '_underscore_names',
 '_uniq',
 '_update_submission',
 'approve',
 'approved_by',
 'archived',
 'author',
 'author_flair_css_class',
 'author_flair_text',
 'banned_by',
 'body',
 'body_html',
 'clear_vote',
 'controversiality',
 'created',
 'created_utc',
 'delete',
 'distinguish',
 'distinguished',
 'downs',
 'downvote',
 'edit',
 'edited',
 'from_api_response',
 'fullname',
 'gild',
 'gilded',
 'has_fetched',
 'id',
 'ignore_reports',
 'is

In [65]:
#Here are the attributes of the Subreddit object
dir(subreddits[0])


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_case_name',
 '_get_json_dict',
 '_has_fetched',
 '_info_url',
 '_listing_urls',
 '_methods',
 '_populate',
 '_post_populate',
 '_underscore_names',
 '_uniq',
 '_url',
 'accept_moderator_invite',
 'add_ban',
 'add_contributor',
 'add_flair_template',
 'add_moderator',
 'add_mute',
 'add_wiki_ban',
 'add_wiki_contributor',
 'clear_all_flair',
 'clear_flair_templates',
 'configure_flair',
 'delete_flair',
 'delete_image',
 'display_name',
 'edit_wiki_page',
 'from_api_response',
 'fullname',
 'get_banned',
 'get_comments',
 'get_contributors',
 'get_controversial',
 'get_controversial_from_all',
